## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [2]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [3]:
# 設定 data_path
dir_data = './data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

### 等寬劃分

In [5]:
app_train['AMT_GOODS_PRICE_CUT'] = pd.cut(app_train['AMT_GOODS_PRICE'],10)
app_train['AMT_GOODS_PRICE_CUT'].value_counts()

(36490.5, 441450.0]       131712
(441450.0, 842400.0]      115261
(842400.0, 1243350.0]      43658
(1243350.0, 1644300.0]     12429
(1644300.0, 2045250.0]      3175
(2045250.0, 2446200.0]       933
(2446200.0, 2847150.0]        30
(2847150.0, 3248100.0]        17
(3649050.0, 4050000.0]        10
(3248100.0, 3649050.0]         8
Name: AMT_GOODS_PRICE_CUT, dtype: int64

#### 等頻劃分

In [9]:
app_train["AMT_GOODS_PRICE_CUT_2"] = pd.qcut(app_train['AMT_GOODS_PRICE'],10)
app_train["AMT_GOODS_PRICE_CUT_2"].value_counts()

(522000.0, 675000.0]      42417
(40499.999, 180000.0]     39484
(373500.0, 450000.0]      35052
(180000.0, 225000.0]      31970
(814500.0, 1093500.0]     30745
(1093500.0, 4050000.0]    30685
(225000.0, 270000.0]      27990
(450000.0, 522000.0]      26475
(270000.0, 373500.0]      23483
(675000.0, 814500.0]      18932
Name: AMT_GOODS_PRICE_CUT_2, dtype: int64

#### 自定義的 bin

In [13]:
app_train["AMT_GOODS_PRICE_CUT_3"] = pd.cut(app_train['AMT_GOODS_PRICE'],[-np.inf,500000,750000,850000,np.inf])
app_train["AMT_GOODS_PRICE_CUT_3"].value_counts()

(-inf, 500000.0]        182618
(850000.0, inf]          60101
(500000.0, 750000.0]     57883
(750000.0, 850000.0]      6631
Name: AMT_GOODS_PRICE_CUT_3, dtype: int64